In [1]:
%pip install langchain chromadb transformers sentence-transformers pypdf langchain-community

Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings

In [3]:
import os

save_path = "../../data/nlp/gsat_170823pdf"
os.system(
    f'curl -o "{save_path}" https://wdr.ubion.co.kr/wowpass/img/event/gsat_170823/gsat_170823.pdf'
)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1223k  100 1223k    0     0  2095k      0 --:--:-- --:--:-- --:--:-- 2095k


0

In [4]:
## pdf 파일로드 하고 쪼개기
loader = PyPDFLoader(save_path)
pages = loader.load_and_split()
print(len(pages))

Ignoring wrong pointing object 2 65536 (offset 0)
Ignoring wrong pointing object 75 65536 (offset 0)
Ignoring wrong pointing object 85 65536 (offset 0)
Ignoring wrong pointing object 94 65536 (offset 0)
Ignoring wrong pointing object 98 65536 (offset 0)
Ignoring wrong pointing object 102 65536 (offset 0)
Ignoring wrong pointing object 109 65536 (offset 0)
Ignoring wrong pointing object 119 65536 (offset 0)
Ignoring wrong pointing object 132 65536 (offset 0)
Ignoring wrong pointing object 148 65536 (offset 0)
Ignoring wrong pointing object 176 65536 (offset 0)
Ignoring wrong pointing object 188 65536 (offset 0)
Ignoring wrong pointing object 192 65536 (offset 0)
Ignoring wrong pointing object 199 65536 (offset 0)
Ignoring wrong pointing object 203 65536 (offset 0)
Ignoring wrong pointing object 207 65536 (offset 0)
Ignoring wrong pointing object 211 65536 (offset 0)
Ignoring wrong pointing object 215 65536 (offset 0)
Ignoring wrong pointing object 219 65536 (offset 0)
Ignoring wrong poi

27


In [5]:
pages[0]

Document(metadata={'source': '../../data/nlp/gsat_170823pdf', 'page': 0}, page_content='2\n01삼성전자 기업분석\n(Samsung Electronics Co., Ltd)\nⅠ 기업 일반 \n1 기업개요\n1) 기업소개 \n본사주소 경기도 수원시 영통구 삼성로 129(매탄동 416)\n사업분야 삼성그룹의 대표 기업으로 휴대폰 , 정보통신기기 , 반도체 , TV 등을 생산 판매하는 제조업체\n홈페이지 www.samsung.com/sec 구분 전기전자 대기업  \n설립일 1961년 07월 01일 대표이사 권오현 \n총자산 1) 244조 매출액 2) 200조\n임직원수 95,374명 \n∙1975년 1월 주식시장 상장\n∙1984년 2월 삼성전자공업주식회사 ->삼성전자주식회사로 사명 변경 \n∙CE(Consumer Electronics), IM(Information technology & Mobile co mmunications), DS(Device Solutions) \n3개의 부문으로 나누어 독립 경영 .\n부문 제품\nCE TV, 모니터 , 냉장고 , 세탁기 , 에어컨 , 프린터 , 의료기기 등\nIM HHP, 네트워크시스템 , 컴퓨터 , 디지털카메라 등\nDS DRAM, NAND Flash, 모바일 AP, LCD패널, OLED패널, LED 등 \n∙주요 사업은 전자전지기계 등 제조 , 전자통신기 등 제도 , 컴퓨터 등 제조 , 반도체 제조 ·조립 등 . 주요 \n제품과 구성비율은 HHP, 네트워크시스템 , 컴퓨터 , 디지털카메라 등 54%, TV, 모니터 , 냉장고 , 세탁기 , \n에어컨 등 2 4 % ,  D R A M ,  N A N D  F l a s h ,  모바일 AP 등 19% 등으로 구분 \n∙지역별로는 본사를 거점으로 한국 및 CE, IM 부문 산하 해외 9개 지역총괄과 DS 부문 산하 해외 5개 \n지역총괄의 생산ㆍ판매법인 등 165개의 동종업종을 영위하는 종속기업으로 구

In [17]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)

splitted_docs = text_splitter.split_documents(pages)
len(splitted_docs)

69

In [18]:
len(splitted_docs[0].page_content)

498

# Embedding

In [19]:
model_huggingface = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

# Chroma

In [22]:
# DB 생성 시 chunk, embedding model 이 들어간다
db = Chroma.from_documents(splitted_docs, model_huggingface)

print("문서의 갯수: ", db._collection.count())

문서의 갯수:  69


In [23]:
# VectorDB 정의
query = "삼성전자의 주요 사업영역은?"
docs = db.similarity_search(query)

for doc in docs:
    print(doc)
    print("--" * 100)

page_content='11
Ⅱ 기업 상세 분석
1 사업분야 (내용)
Q1삼성전자의 대표적 사업분야에 대해 설명할 수 있습니까 ?
A
삼성전자는 크게 CE(Consumer Electronics) 사업부문 , IM(Information technology & Mobile communica-
tion) 사업부문 , DS(Device Solutions) 사업부문 등 3개 사업부문으로 나누어 독립 경영을 합니다 .
⑴ Consumer Electronics (CE) 부문 
① 영상디스플레이 : 진화하는 스마트 TV, 초대형 프리미엄 TV 전략으로 8년 연속 세계 1위에 
도전
② 생활가전 : 새로운 기술과 가치 창출로 생활과 문화를 바꾸는 혁신을 준비
③ 의료기기 : 정확하고 빠른 진단을 도와주는 새롭고 혁신적인 의료기기를 개발
⑵ Information technology & Mobile communication (IM) 부문' metadata={'page': 9, 'source': '../../data/nlp/gsat_170823pdf'}
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
page_content='2
01삼성전자 기업분석
(Samsung Electronics Co., Ltd)
Ⅰ 기업 일반 
1 기업개요
1) 기업소개 
본사주소 경기도 수원시 영통구 삼성로 129(매탄동 416)
사업분야 삼성그룹의 대표 기업으로 휴대폰 , 정보통신기기 , 반도체 , TV 등을 생산 판매하는 제조업체
홈페이지 www.samsung.com/sec 구분 전기전자 대기업  
설립일 1961년 07월 01일 대표이사 권오현 
총자산 1) 244조 매출액 2) 200조
임

In [24]:
# 벡터화된 텍스트를 Chroma DB 파일로 저장
db_toFiles = Chroma.from_documents(
    splitted_docs, model_huggingface, persist_directory="../../data/nlp/db/samsung.db"
)

print("문서의 수 : ", db_toFiles._collection.count())

문서의 수 :  69


In [26]:
# DB 불러오기
db_fromFile = Chroma(
    persist_directory="../../data/nlp/db/samsung.db",
    embedding_function=model_huggingface,
)

docs = db_fromFile.similarity_search(query)
docs[0].page_content

'11\nⅡ 기업 상세 분석\n1 사업분야 (내용)\nQ1삼성전자의 대표적 사업분야에 대해 설명할 수 있습니까 ?\nA\n삼성전자는 크게 CE(Consumer Electronics) 사업부문 , IM(Information technology & Mobile communica-\ntion) 사업부문 , DS(Device Solutions) 사업부문 등 3개 사업부문으로 나누어 독립 경영을 합니다 .\n⑴ Consumer Electronics (CE) 부문 \n① 영상디스플레이 : 진화하는 스마트 TV, 초대형 프리미엄 TV 전략으로 8년 연속 세계 1위에 \n도전\n② 생활가전 : 새로운 기술과 가치 창출로 생활과 문화를 바꾸는 혁신을 준비\n③ 의료기기 : 정확하고 빠른 진단을 도와주는 새롭고 혁신적인 의료기기를 개발\n⑵ Information technology & Mobile communication (IM) 부문'

## Chroma 문서 추가
* 저장된 문서들은 각각 고유한 id 값을 가진다
* 새로운 문서를 추가할 때 id 를 부여하는 것도 가능

In [27]:
# 상위 5개의 id 출력
db_fromFile._collection.get()["ids"][:5]

['ac5cd3af-87d7-4736-a923-e23b40d8c78c',
 '42b4b7a0-04a2-4f79-bd67-28fbfa4d5058',
 'c44fadb5-d5bd-4558-ac63-979198b70b8b',
 '3aaa5290-624c-4fde-94dd-78a258b2613b',
 '71bb8ffc-3d06-4708-8a36-730abfe48bd6']

In [28]:
# 신규 문서 추가하기
text1 = """삼성전자가 미국 반도체업체 엔비디아에 고대역폭 메모리(HBM)를 납품하기 위한 테스트를 아직 통과하지 못했다고 로이터통신이 3명의 익명 소식통을 인용해 24일 보도했다."""
text2 = """엔비디아발 훈풍으로 전 세계 반도체가 들썩이는 가운데 국내 증시에서는 SK하이닉스 (198,600원 ▼1,400 -0.70%)의 활약이 돋보인다. 엔비디아에 대한 투자자들의 기대치가 높아지고 있어 SK하이닉스 역시 상승세를 이어갈 것이란 전망이 쏟아진다. 다만 SK하이닉스 주가에 상승동력으로 작용했던 엔비디아 실적 재료가 소진된 점은 차익실현 매물이 출현할 가능성을 높인다."""

In [29]:
# 반드시 VectorStore 에 추가되어야 할 형식은 document
# document 는 metadata, page_content 로 구성

from langchain.docstore.document import Document

# metadata : 데이터를 설명하기 위한 데이터
new_doc = Document(
    page_content=text2, metadata={"source": "https://news.naver.com", "pages": 10}
)

new_doc

Document(metadata={'source': 'https://news.naver.com', 'pages': 10}, page_content='엔비디아발 훈풍으로 전 세계 반도체가 들썩이는 가운데 국내 증시에서는 SK하이닉스 (198,600원 ▼1,400 -0.70%)의 활약이 돋보인다. 엔비디아에 대한 투자자들의 기대치가 높아지고 있어 SK하이닉스 역시 상승세를 이어갈 것이란 전망이 쏟아진다. 다만 SK하이닉스 주가에 상승동력으로 작용했던 엔비디아 실적 재료가 소진된 점은 차익실현 매물이 출현할 가능성을 높인다.')

In [30]:
new_docs = [new_doc]

# 문서 추가하기
id = db_fromFile.add_documents(new_docs, ids=["7"])

id

['7']

In [31]:
db_fromFile._collection.get(id)

{'ids': ['7'],
 'embeddings': None,
 'metadatas': [{'pages': 10, 'source': 'https://news.naver.com'}],
 'documents': ['엔비디아발 훈풍으로 전 세계 반도체가 들썩이는 가운데 국내 증시에서는 SK하이닉스 (198,600원 ▼1,400 -0.70%)의 활약이 돋보인다. 엔비디아에 대한 투자자들의 기대치가 높아지고 있어 SK하이닉스 역시 상승세를 이어갈 것이란 전망이 쏟아진다. 다만 SK하이닉스 주가에 상승동력으로 작용했던 엔비디아 실적 재료가 소진된 점은 차익실현 매물이 출현할 가능성을 높인다.'],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents']}

In [34]:
# 문서 추가 후 갯수 확인
db_fromFile._collection.count()

70

## Chroma 문서 삭제

In [35]:
db_fromFile._collection.delete(ids=id)
db_fromFile._collection.get(id)

{'ids': [],
 'embeddings': None,
 'metadatas': [],
 'documents': [],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents']}

# FAISS

In [36]:
from langchain.vectorstores import FAISS

faiss_db = FAISS.from_documents(splitted_docs, model_huggingface)

In [37]:
docs = faiss_db.similarity_search(query)
docs[0]

Document(metadata={'source': '../../data/nlp/gsat_170823pdf', 'page': 9}, page_content='11\nⅡ 기업 상세 분석\n1 사업분야 (내용)\nQ1삼성전자의 대표적 사업분야에 대해 설명할 수 있습니까 ?\nA\n삼성전자는 크게 CE(Consumer Electronics) 사업부문 , IM(Information technology & Mobile communica-\ntion) 사업부문 , DS(Device Solutions) 사업부문 등 3개 사업부문으로 나누어 독립 경영을 합니다 .\n⑴ Consumer Electronics (CE) 부문 \n① 영상디스플레이 : 진화하는 스마트 TV, 초대형 프리미엄 TV 전략으로 8년 연속 세계 1위에 \n도전\n② 생활가전 : 새로운 기술과 가치 창출로 생활과 문화를 바꾸는 혁신을 준비\n③ 의료기기 : 정확하고 빠른 진단을 도와주는 새롭고 혁신적인 의료기기를 개발\n⑵ Information technology & Mobile communication (IM) 부문')

## FAISS DB 파일 저장 & 불러오기

In [38]:
faiss_db.save_local("../../data/nlp/db/faiss_index")

In [39]:
faiss_fromFile = faiss_db.load_local(
    "../../data/nlp/db/faiss_index",
    model_huggingface,
    allow_dangerous_deserialization=True,
)

docs = faiss_fromFile.similarity_search(query)
docs[0]

Document(metadata={'source': '../../data/nlp/gsat_170823pdf', 'page': 9}, page_content='11\nⅡ 기업 상세 분석\n1 사업분야 (내용)\nQ1삼성전자의 대표적 사업분야에 대해 설명할 수 있습니까 ?\nA\n삼성전자는 크게 CE(Consumer Electronics) 사업부문 , IM(Information technology & Mobile communica-\ntion) 사업부문 , DS(Device Solutions) 사업부문 등 3개 사업부문으로 나누어 독립 경영을 합니다 .\n⑴ Consumer Electronics (CE) 부문 \n① 영상디스플레이 : 진화하는 스마트 TV, 초대형 프리미엄 TV 전략으로 8년 연속 세계 1위에 \n도전\n② 생활가전 : 새로운 기술과 가치 창출로 생활과 문화를 바꾸는 혁신을 준비\n③ 의료기기 : 정확하고 빠른 진단을 도와주는 새롭고 혁신적인 의료기기를 개발\n⑵ Information technology & Mobile communication (IM) 부문')

In [41]:
# 저장된 문서 갯수 확인하기
faiss_db.index.ntotal

69

In [43]:
# 저장된 문서들 출력
faiss_db.docstore.__dict__

{'_dict': {'1bcdddb8-3212-4b60-b59d-1cc970cbf02e': Document(metadata={'source': '../../data/nlp/gsat_170823pdf', 'page': 0}, page_content='2\n01삼성전자 기업분석\n(Samsung Electronics Co., Ltd)\nⅠ 기업 일반 \n1 기업개요\n1) 기업소개 \n본사주소 경기도 수원시 영통구 삼성로 129(매탄동 416)\n사업분야 삼성그룹의 대표 기업으로 휴대폰 , 정보통신기기 , 반도체 , TV 등을 생산 판매하는 제조업체\n홈페이지 www.samsung.com/sec 구분 전기전자 대기업  \n설립일 1961년 07월 01일 대표이사 권오현 \n총자산 1) 244조 매출액 2) 200조\n임직원수 95,374명 \n∙1975년 1월 주식시장 상장\n∙1984년 2월 삼성전자공업주식회사 ->삼성전자주식회사로 사명 변경 \n∙CE(Consumer Electronics), IM(Information technology & Mobile co mmunications), DS(Device Solutions) \n3개의 부문으로 나누어 독립 경영 .\n부문 제품\nCE TV, 모니터 , 냉장고 , 세탁기 , 에어컨 , 프린터 , 의료기기 등'),
  '86b50ae4-76c8-447b-96f4-f4e56ca41e9b': Document(metadata={'source': '../../data/nlp/gsat_170823pdf', 'page': 0}, page_content='IM HHP, 네트워크시스템 , 컴퓨터 , 디지털카메라 등\nDS DRAM, NAND Flash, 모바일 AP, LCD패널, OLED패널, LED 등 \n∙주요 사업은 전자전지기계 등 제조 , 전자통신기 등 제도 , 컴퓨터 등 제조 , 반도체 제조 ·조립 등 . 주요 \n제품과 구성비율은 HHP, 네트워크시스템 , 컴퓨터 , 디지털카메라 등 54%, TV, 모니터 , 냉장고 ,

## FAISS 문서 삭제

In [44]:
faiss_db.delete(["1bcdddb8-3212-4b60-b59d-1cc970cbf02e"])

True

In [45]:
faiss_db.index.ntotal

68

## 신규 문서 추가

In [46]:
# 새로운 데이터를 위한 vector space 생성
new_db = FAISS.from_texts([text1, text2], model_huggingface)

In [47]:
# 새롭게 만들어진 vector store 를 기존 vector store 와 병합
faiss_db.merge_from(new_db)

In [48]:
faiss_db.index.ntotal

70